# Twitch Dashboard - finished code

Team Project initially taken on by
+ Vardayini Sharma - https://www.linkedin.com/in/vardayini/
+ Maddhujeet Chandra - https://www.linkedin.com/in/maddhujeet/
+ Kaivalya Powale - https://www.linkedin.com/in/kaivalya-powale/

GitHub repository- https://github.com/kaivalyapowale/Twitch-Dashboard/

### Importing Libraries

In [ ]:
import json
import requests
import pandas as pd
from pandas.io.json import json_normalize
import time
import threading

### Authentication

Put in your persinal client id and client secret

In [ ]:
#Client ID: --***---
#Client Secret: ---***---
#Client ID and Client Secret are sensititve and you should not share them

client_id= <yourclientid>
client_secret= <yourclientsecret>

#Request for the access code using requests library
#I have chosen this method of authentication with my goal in mind

access_code = requests.post('https://id.twitch.tv/oauth2/token?client_id='+str(client_id)+'&client_secret='+str(client_secret)+'&grant_type=client_credentials')

#access token response is a JSON-encoded app access token
access_token = json.loads(access_code.text)
access_token = access_token['access_token']

#Sample response is
"""
{
  "access_token": "prau3ol6mg5glgek8m89ec2s9q5i3i",
  "refresh_token": "",
  "expires_in": 3600,
  "scope": [],
  "token_type": "bearer"
}
"""

### Recurring code for the dashboard

Just mention the filepath where you want the csv to be saved.

In [ ]:
def twitch():
    threading.Timer(60.0, twitch).start()
    
    # Top Games
    headers = {
        'Authorization' : 'Bearer '+str(access_token),
    }
    games_response = requests.get('https://api.twitch.tv/helix/games/top?first=100', headers=headers)

    games_response_json = json.loads(games_response.text)
    topgames_data = games_response_json['data']
    topgames_df = pd.DataFrame.from_dict(json_normalize(topgames_data), orient='columns')
    
    # Top Streamers
    headers = {
        'Authorization' : 'Bearer '+str(access_token),
    }
    topstreamsforgame_response = requests.get('https://api.twitch.tv/helix/streams?game_id='+str(topgames_df['id'][0])+'&first=25', headers=headers)

    topstreamsforgame_response_json = json.loads(topstreamsforgame_response.text)
    topstreamsforgame_data = topstreamsforgame_response_json['data']
    topstreamsforgame_df = pd.DataFrame.from_dict(json_normalize(topstreamsforgame_data), orient='columns')

    for i in range(1,19) :
    
        headers = {
            'Authorization' : 'Bearer '+str(access_token),
        }
        topstreamsforgame_response = requests.get('https://api.twitch.tv/helix/streams?game_id='+str(topgames_df['id'][i])+'&first=25', headers=headers)
    
        topstreamsforgame_response_json = json.loads(topstreamsforgame_response.text)
        topstreamsforgame_data = topstreamsforgame_response_json['data']
        topstreamsforgame_df_temp = pd.DataFrame.from_dict(json_normalize(topstreamsforgame_data), orient='columns')
    
        frames = [topstreamsforgame_df, topstreamsforgame_df_temp]
        topstreamsforgame_df = pd.concat(frames, ignore_index=True)

    # Now that the FOR loop is exited and we have all our data, we export it into a csv
    
    export_topgames_csv = topgames_df.to_csv (r'/Users/kaivalya/Twitch/Sample/topgames.csv', index = None, header=True) #Don't forget to add '.csv' at the end of the path
    
    export_topstreamsforgame_csv = topstreamsforgame_df.to_csv (r'/Users/kaivalya/Twitch/Sample/topstreamsforgames.csv', index = None, header=True)
    
# Our function is defined and it overwrites the CSV every 60 seconds. Now, we call it.
twitch()